# TP2 - scaling, différences finies et méthode du gradient

In [4]:
%pylab inline

from scipy.optimize import minimize 
import numpy as np 

Populating the interactive namespace from numpy and matplotlib


# Exercice 1 - intérêt de la mise à l'échelle ("scaling") en optimisation

Considérons la fonction de deux variables réelles suivante :
$$
f_{M}(x,y) = (x-M)^4+(x-M)^2+\left(y-\dfrac{1}{M}\right)^4+\left(y-\dfrac{1}{M}\right)^2.
$$

+ En utilisant la fonction *minimize* de la bibliothèque *scipy.optimize*, chercher une solution approchée $(x^*,y^*)$ de la minimisation de cette fonction $f_{M}$ en partant du point $(1,1)$ et en utilisant l'option *Nelder-Mead*.
Observer ce qui se passe lorsque $M >> 1$, par exemple $M = 10^9$.

+ Ensuite, construire la fonction $f^S_M$ définie par 
$$
f^S_{M}(u,v) = f_M\left(Mu,\dfrac{v}{M}\right)
$$
et chercher une solution approchée $(u^*,v^*)$ de la minimisation de cette fonction $f^S_{M}$ en partant du point $\left(\dfrac{1}{M},M\right)$.

+ Comparer $(x^*,y^*)$ et $\left(M u^*,\dfrac{v^*}{M}\right)$.

Remarque : on ajoutera *options={'disp': True, 'maxiter':1000}* dans les appels à *minimize*.

In [35]:
M = 10**9
f = lambda a : (a[0]-M)**2 + (a[0]-M)**4 + (a[1]-1/M)**2 + (a[1]-1/M)**4
fs = lambda b : (M*(b[0]-1))**2 + (M*(b[0]-1))**4 + ((b[1]-1)/M)**2 + ((b[1]-1)/M)**4 

In [36]:
x0 = np.array([1,1])

res_f = minimize(f, x0, method='Nelder-Mead',options={'disp': True, 'maxiter':1000})

(x_sol_f,y_sol_f) = np.array(res_f.x) 

print(x_sol_f,y_sol_f)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 260
         Function evaluations: 467
999999999.9999704 1.2911399266203106e-05


In [37]:
x0 = np.array([1/M,M])

res_fs = minimize(fs, x0, method='Nelder-Mead',options={'disp': True, 'maxiter':1000})

(u_sol_fs,v_sol_fs) = np.array(res_fs.x) 

print(M*u_sol_fs,v_sol_fs/M)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 309
         Function evaluations: 557
1000000000.0 1.0000110451478519e-09


la solution est plus exacte après la mise en échelle au coup de plus 

# Exercice 2 - approximation d'un gradient et d'une matrice hessienne par différences finies

On se rappelle qu'une approximation de la dérivée $f'(x)$ d'une fonction $f$ définie, continue et dérivable sur $\mathbb{R}$ est simplement, pour $h$ petit, 
$a_h := \dfrac{f(x+h)-f(x)}{h}.$
On a alors $|a_h-f'(x)| = O(h)$.

On généralise aisément cela au cas d'une fonction $f$ à valeurs réelles définie sur $\mathbb{R}^n$. En effet, si on note $e_i$ le $i$-ème vecteur de la base canonique de $\mathbb{R}^n$, la $i$-ème composante de $\nabla f(x)$ est approchée par
$$
\dfrac{f(x+he_i)-f(x)}{h}.
$$

De manière analogue, on peut approcher la composante $(i,j)$ de la matrice hessienne de $f$ en $x$ par 
$$
\dfrac{f(x+he_i+he_j)-f(x+he_i)-f(x+he_j)+f(x)}{h^2}.
$$

+ Ecrire une fonction **FDgrad(f,x,h)** qui calcule une approximation du gradient de la fonction $f$ au point $x$ en prenant un pas $h$.

+ Tester cette fonction pour une fonction $f$ dont vous connaissez le gradient (par exemple la fonction de Rosenbrock) et observez son comportement lorsque $h$ varie dans l'ensemble $\left\{10^{-k}, k = 1, 2, ... 16\right\}$. Pour ce faire, on pourra faire afficher l'erreur commise par rapport au gradient exact.

+ Ecrire une fonction **FDhess(f,x,h)** qui calcule une approximation de la matrice hessienne de la fonction $f$ au point $x$ en prenant un pas $h$.

+ Tester cette fonction pour une fonction $f$ dont vous connaissez la matrice hessienne et observez son comportement lorsque $h$ varie.

# Exercice 3 - méthode du gradient à pas fixe

Lorsqu'on cherche à minimiser une fonction $f$ définie sur $\mathbb{R}^n$, l'une des méthodes les plus élémentaires est la méthode de gradient. Elle consiste à construire une suite d'itérés $(x_k)_{k\in \mathbb{N}}$ de la manière suivante : partant d'un $x_0 \in \mathbb{R}^n$, pour tout $k \in \mathbb{N}$,
$$
x_{k+1} = x_k+\alpha_k d_k, \mbox{ avec } d_k = -\nabla f(x_k).
$$

Comme $d_k = -\nabla f(x_k)$ est une direction de descente ($\nabla f(x_k)^\top d_k < 0$), la suite ci-dessus fait nécessairement décroître $f$, quitte à prendre un pas constant $\alpha_k  = \alpha$ très petit.

+ Ecrire une fonction **GPF(x0,f,df,alpha,itmax)** qui fournit la suite décrite ci-dessus pour minimiser une fonction **f**, en partant du vecteur initial **x0** et en utilisant le pas fixe stocké dans **alpha**, sans faire plus de **itmax** itérations. L'argument **df** est le nom de la fonction qui calcule le gradient de **f**.

+ Pour mettre au point cette fonction, on commencera par faire un nombre fixe d'itérations et on la testera sur la fonction de Rosenbrock dont on connaît le minimum :
$$
p(x_1^2-x_2)^2+(1-x_1)^2.
$$
On pourra faire les essais avec $p = 10$.
Les points de départ standards pour cette fonction sont $(-1.2, 1)$ et $(6.39,-0.221)$.
Le temps de déboguer, on prendra **itmax** = 100, puis ensuite on le mettra à 1000.
Il faudra prendre le temps de choisir les pas fixes adaptés à chacun des deux points de départ.

Afin de constater que la méthode fonctionne, 
+ on fera un tracé des itérés construits, superposé aux contours de la fonction minimisée,
+ on fera un tracé en échelle logarithmique de la décroissance de la fonction $f$ en fonction des itérations.

# Exercice 4 - recherche linéaire d'Armijo

On a observé ci-dessus que le pas fixe n'est pas forcément très pratique. De fait, il est rarement utilisé en pratique.

On utilise plutôt une **recherche linéaire** pour trouver un pas $\alpha_k$ adapté à chaque itération. Par souci de simplicité, on décrit ici une recherche linéaire basée sur 
+ la règle d'Armijo : on cherche un pas $\alpha_k$ qui vérifie la condition de décroissance suffisante
$$
f(x_k+\alpha_k d_k) \leq f(x_k) + c_1 \alpha_k \nabla f(x_k)^\top d_k,
$$
avec $c_1 \in \left]0,\dfrac{1}{2}\right[$. Une valeur classique $c_1$ est $10^{-4}$ ; 
+ un "backtracking" : on cherche $\alpha_k$ parmi les $\left\{\dfrac{1}{\beta^k}, k \in\mathbb{N}\right\}$.

On se propose de coder l'algorithme de recherche linéaire suivant dans une fonction 
**RLA(xk,dk,alpha0,f,ps)** : étant donnés l'itéré courant $x_k$, la direction de descente $d_k$, la fonction $f$ à minimiser et le produit scalaire **ps** contenant $\nabla f(x_k)^\top d_k$, on initialise le pas $\alpha$ à $\alpha_0$, puis tant que la condition d'Armijo n'est pas satisfaite, on divise $\alpha$ par $2$. La fonction **RLA** doit retourner la valeur du pas ainsi obtenue.

# Exercice 5 - méthode de gradient globalisé

A partir de la fonction GPF(x0,f,df,alpha,itmax), écrire une fonction **GRLA(x0,f,df,alpha0,itmax)** qui fournit la suite 
$$
x_{k+1} = x_k+\alpha_k d_k, \mbox{ avec } d_k = -\nabla f(x_k).
$$
pour minimiser une fonction **f**, en partant du vecteur initial **x0**, mais cette fois en utilisant la recherche linéaire d'Armijo codée ci-dessus, initialisée à **alpha0** pour ajuster le pas $\alpha_k$ à chaque itération. 

On fera les mêmes tests que dans l'exercice 3, en initialisant la recherche linéaire avec $\alpha_0 = 1$ et on ajoutera le tracé de l'évolution du pas $\alpha$ au cours des itérations.

# Exercice 6 - amélioration des critères d'arrêt

Maintenant que la fonction **GRLA** est au point, on peut essayer de mettre en place des critères d'arrêt plus réalistes :  
$$
k > itmax, 
\quad\mbox{ou}\quad
\max_{1\leq i\leq n}\left|
\dfrac{(\nabla f(x_k))_i (x_{k+1})_i}{f(x_{k+1})}
\right| 
\leq \varepsilon_g, 
\quad\mbox{ou}\quad
\max_{1\leq i\leq n}\left|
\dfrac{(x_{k+1})_i-(x_{k})_i}{(x_{k+1})_i}
\right| 
\leq \varepsilon_x.
$$

Ainsi, à partir de GRLA(x0,f,df,alpha0,itmax), construire une fonction **GRLA2(x0,f,df,alpha0,itmax,epsilg,epsilx)** qui remplace la boucle *for* en une boucle *while* avec les critères d'arrêt indiqués ci-dessus.

Refaire les mêmes tests que précédemment, en commençant avec $\varepsilon_g = \varepsilon_x = 10^{-4}$.

Remarque : en pratique, pour éviter les problèmes lorsque l'un des $(x_{k+1})_i$ ou $f(x_{k+1})$ devient nuls, on utilise plutôt les tests robustes suivants : 
$$
k > itmax, 
\quad\mbox{ou}\quad
\max_{1\leq i\leq n}\left|
\dfrac{(\nabla f(x_k))_i \max\{|(x_{k+1})_i|,typx_i\}}{\max\{|f(x_{k+1})|,typf\}}
\right| 
\leq \varepsilon_g, 
\quad\mbox{ou}\quad
\max_{1\leq i\leq n}\left|
\dfrac{(x_{k+1})_i-(x_{k})_i}{\max\{|(x_{k+1})_i|,typx_i\}}
\right| 
\leq \varepsilon_x.
$$